## Замечания:
Актуально:
* С дисбалансом из-за полупустых классов можно бороться либо удалением маленьких классов, либо **пересемплированием**. Пока выбираю второй вариант, но в любом случае чудес ждать от предсказания маленьких по объему классов не стоит - если в обучающей выборке есть 1 строка какого-то класса, значит он будет предсказывать этот класс исключительно при полном соответствии тестовых строк данной одной строке. 
*


Актуально для меня:
* Можно улучшить обработку текстовых данных (можно попробовать в сыром виде запихать в кэтбуст)
* Если лемматизация будет работать долго (есть такая вероятность) - можно поменять на стеммер

Старое:
* Время работы? - **нужно протестировать на больших датасетах**
* Полупустые классы из-за которых возникает дисбаланс - **бороться**

* Вариант с тем, что в одной колонке название характеристики, в другой значение будет работать плохо (вот пример), т.к. модели без разницы на порядок следования колонок
* Правильно ли я понимаю, что все колонки кроме ХК 1 и целевой имеют тип данных String?
* **Как предсказывать строки с пустыми значениями во всех колонках ХК? (может их сразу откидывать)?**
* Названия первой колонки должны быть всегда одинаковые
* !!!Важно!!! Будем заменять числовые факторы на категориальные, если в них маленькое количество уникальных значений или одно значение встречается очень часто
* Были ошибки в названиях колонок: 'ХК_ка т_01'

**Проблемы, решение которых нужно будет автоматизировать:**
* Несбалансированность классов
* Пропуски в данных
* Автоматическое кодирование текстовых столбцов

In [28]:
#!pip install loguru
#!pip install imblearn
#!pip install pymystem3
#!pip install catboost

In [3]:
import pandas as pd
import numpy as np
from loguru import logger
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer

#nltk.download('punkt')
#nltk.download('stopwords')

#### Глобальные переменные

In [4]:
# Максимальное количество уникальных значений для категориального фактора, при котором он может обрабатываться методом one-hot encoding (добавится максимум столько столбцов)
OneHotEncodingLimit = 30

### Работа с данными

In [5]:
df = pd.read_excel('data/tire_classificator_data.xlsx')

In [6]:
df = df[df['ID класса (ТАРГЕТ)'].notna()]

In [7]:
df.head(1)

,ID класса (ТАРГЕТ),Наименование терминального класса,Код родительского класса,Наименование родительского класса,Историческое наименование,ХК_Кат_01,Значение ХК_Кат_01,ХК_Кат_02,Значение ХК_Кат_02,ХК_Кат_03,Значение ХК_Кат_03,ХК_Кат_04,Значение ХК_Кат_04,ХК_Кат_05,Значение ХК_Кат_05,ХК_Стр_01,Значение ХК_Стр_01,ХК_Булево_01,Значение ХК_Булево_01
0,12388594,Шины автомобильные зимние 215/75 R16,01.09.08.01,ШИНЫ АВТОМОБИЛЬНЫЕ,"Шины автомобильные зимние 215/75, R16, 116/114...",Производитель,GoodYear,Диаметр,R16,Размерность,215/75,Сезоность,зимние,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
target = df['ID класса (ТАРГЕТ)']

# Удалим все лишние текстовые столбцы кроме "Исторического наименования"
trainset_columns = []
for column in df.columns:
    if (column == 'Историческое наименование') or (re.fullmatch(r'ХК_.*', column)!=None) or (re.fullmatch(r'Значение.*', column)!=None):
        trainset_columns.append(column)

factors_df = df[trainset_columns]

#### Работаем с типами данных столбцов

In [9]:
def format_column_types(columns: list):
    '''
    Обрабатывает названия колонок из массива columns.
    Возвращает словарь с парами: название колонки - ее тип данных  
    '''
    feature_types_dict = {}
    for column in columns:
        type_pattern = r'ХК_([^_]+)_.*'
        if column[0:2] == 'ХК':
            feature_types_dict[column] = 'Кат'
        elif column[0:8]=='Значение':
            column_type = re.findall(type_pattern, column)[0]
            feature_types_dict[column] = column_type
        else:
            feature_types_dict[column] = 'Стр'
    return feature_types_dict

feature_types_dict = format_column_types(factors_df.columns)

In [10]:
def check_number_to_categorical(column: str, factor: pd.Series):
    logger.info(f'Начинаем проверку численного фактора {column}\n')
    logger.debug(f'Размер фактора:{factor.size}')
    logger.debug(f'Количество уникальных значений: {factor.drop_duplicates().size}')
    logger.debug(f'Процент заполненности фактора: {factor[factor.notnull()].size / factor.size * 100}%')
    popular_value = pd.DataFrame(factor.value_counts().sort_values(ascending=False).head(1)/factor[factor.notnull()].size*100)
    popular_value.columns = ['Частота']
    logger.debug(f'Cамое частое значение фактора: \n{popular_value}')
    
    if float(popular_value.iloc[0])>=50:
        logger.info(f'Переводим числовой фактор {column} в категориальный')
        return True
    
    
for feature in feature_types_dict.keys():
    if feature_types_dict.get(feature) == 'Стр':
        #logger.debug(f'Строковый фактор: {feature}')
        #todo ДОДЕЛАТЬ пока ничего не делаем, чтобы не потерять пропущенные значения при преобразовании в строковый формат
        factors_df_copy = factors_df.copy()
        factors_df[feature] = factors_df_copy[feature].astype(object)
    elif feature_types_dict.get(feature) == 'Булево':
        factors_df_copy = factors_df.copy()
        factors_df[feature] = factors_df_copy[feature].astype(bool)
    elif feature_types_dict.get(feature) == 'Числ':
        if check_number_to_categorical(feature, factors_df[feature]):
            feature_types_dict[feature] = 'Кат'
            factors_df_copy = factors_df.copy()
            factors_df[feature] = factors_df_copy[feature].astype(object)
        else:
            factors_df_copy = factors_df.copy()
            factors_df[feature] = factors_df_copy[feature].astype(float)
    elif feature_types_dict.get(feature) == 'Кат':
        logger.debug(f'Категориальный фактор: {feature}')
        #todo ДОДЕЛАТЬ преобразование категориальных колонок (пока не делаем, т.к. возможно будет catboost)


C:\Users\User\AppData\Local\Temp\ipykernel_6180\1022614043.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors_df[feature] = factors_df_copy[feature].astype(object)
2023-10-12 14:08:02.217 | DEBUG    | __main__:<module>:33 - Категориальный фактор: ХК_Кат_01
2023-10-12 14:08:02.218 | DEBUG    | __main__:<module>:33 - Категориальный фактор: Значение ХК_Кат_01
2023-10-12 14:08:02.219 | DEBUG    | __main__:<module>:33 - Категориальный фактор: ХК_Кат_02
2023-10-12 14:08:02.219 | DEBUG    | __main__:<module>:33 - Категориальный фактор: Значение ХК_Кат_02
2023-10-12 14:08:02.220 | DEBUG    | __main__:<module>:33 - Категориальный фактор: ХК_Кат_03
2023-10-12 14:08:02.220 | DEBUG    | __main__:<module>:33 - Категориальный фактор: Значение ХК_Кат_

#### Заполняем пропуски в данных
В зависимости от типа данных колонки заполняем пропуски по-разному:
*   Стр -  т.к. переводим строки в числа, то пропущенные значение пусть будут = 0
*   Числ - #todo По умолчанию = 0. Если присутствует значение, количество которого в заполненных строках >=50% => то фактор станет категориальным, а не численным. 
*   Булево - #todo будем считать, что у нас всегда такие столбцы отвечают на вопрос: "Есть что-то? - Да/Нет". Если нет ответа => Нет
*   Кат - 'Emptyclass'

In [11]:
factors_df.isna().sum()

Историческое наименование     0
ХК_Кат_01                     0
Значение ХК_Кат_01            0
ХК_Кат_02                     0
Значение ХК_Кат_02            0
ХК_Кат_03                     0
Значение ХК_Кат_03            0
ХК_Кат_04                     0
Значение ХК_Кат_04            0
ХК_Кат_05                    40
Значение ХК_Кат_05           40
ХК_Стр_01                    49
Значение ХК_Стр_01           49
ХК_Булево_01                 40
Значение ХК_Булево_01         0
dtype: int64

In [27]:
not_empty_factors_df = factors_df.copy()
for column in not_empty_factors_df.columns:
    if feature_types_dict.get(column) == 'Кат':
        not_empty_factors_df.loc[not_empty_factors_df[column].isna(), column] = f'EmptyValue'
    elif feature_types_dict.get(column) == 'Числ':
        not_empty_factors_df.loc[not_empty_factors_df[column].isna(), column] = 0
    elif feature_types_dict.get(column) == 'Стр':
        not_empty_factors_df.loc[not_empty_factors_df[column].isna(), column] = ''
    elif feature_types_dict.get(column) == 'Булево':
        not_empty_factors_df.loc[not_empty_factors_df[column].isna(), column] = 0


In [13]:
not_empty_factors_df.isna().sum().sort_values()

Историческое наименование    0
ХК_Кат_01                    0
Значение ХК_Кат_01           0
ХК_Кат_02                    0
Значение ХК_Кат_02           0
ХК_Кат_03                    0
Значение ХК_Кат_03           0
ХК_Кат_04                    0
Значение ХК_Кат_04           0
ХК_Кат_05                    0
Значение ХК_Кат_05           0
ХК_Стр_01                    0
Значение ХК_Стр_01           0
ХК_Булево_01                 0
Значение ХК_Булево_01        0
dtype: int64

#### Кодируем строковые переменные
Возможные варианты:
* bag_of_words - пока остановимся на нем
* tf_idf 

In [28]:
russian_stopwords = stopwords.words("russian")
mystem = Mystem()

def text_preprocessing(text):
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "  and token.strip() not in punctuation]
    tokens = ' '.join(tokens)
    return tokens

def text_feature_preprocessing(text_feature):
    '''
    Функция преобразования текстовых факторов
    - Переводим в нижний регистр
    - Удаляем знаки препинания
    - Удаляем стоп слова
    - Проводим лемматизацию
    '''
    processed_feature = []
    text_feature = text_feature.replace(r'[^\w\s]',' ', regex=True).replace(r'\s+',' ', regex=True).str.lower()
    #processed_text_feature = text_feature.apply(text_preprocessing)
    #return processed_text_feature
    return text_feature

def handle_text_feature(text_feature: pd.Series):
    '''
    Функция обработки строкового фактора:
    - Проводим препроцессинг
    - Формируем "Мешок строк" (bag of words)
    '''
    text_feature = text_feature.drop_duplicates()
    processed_text_feature = text_feature_preprocessing(text_feature)
    vectorizer = CountVectorizer()
    vectorizer.fit(processed_text_feature)
    vectorized_text_feature = pd.DataFrame(vectorizer.transform(processed_text_feature).toarray())

    # Удалим неинформативные столбцы
    informative_word_columns = vectorized_text_feature.sum()[
            (vectorized_text_feature.sum()>=vectorized_text_feature.shape[1]*0.01) &
            (vectorized_text_feature.sum()!=vectorized_text_feature.shape[1])
        ].index 
    handled_text_feature = vectorized_text_feature[informative_word_columns]
    handled_text_feature.columns = pd.Series(vectorizer.get_feature_names_out())[informative_word_columns]
    return handled_text_feature

In [29]:
from sklearn.preprocessing import LabelEncoder

def handle_cat_feature(cat_feature: pd.Series):
    cat_feature = cat_feature.astype(str)
    unique_values_count = cat_feature.drop_duplicates().size
    if unique_values_count <= OneHotEncodingLimit:
        #OneHotEncoding
        cat_feature_encoded = pd.get_dummies(cat_feature)
    else:
        #LabelEncoding - чтобы сильно не увеличивать количество факторов
        le = LabelEncoder()
        cat_feature_encoded = pd.DataFrame(le.fit_transform(cat_feature))
    return cat_feature_encoded

In [34]:
copy_df.isna().sum()

Историческое наименование    0
ХК_Кат_01                    0
Значение ХК_Кат_01           0
ХК_Кат_02                    0
Значение ХК_Кат_02           0
ХК_Кат_03                    0
Значение ХК_Кат_03           0
ХК_Кат_04                    0
Значение ХК_Кат_04           0
ХК_Кат_05                    0
Значение ХК_Кат_05           0
ХК_Стр_01                    0
Значение ХК_Стр_01           0
ХК_Булево_01                 0
Значение ХК_Булево_01        0
dtype: int64

In [35]:
trainset = pd.DataFrame()
copy_df = not_empty_factors_df.copy()

for feature in feature_types_dict:
    if feature_types_dict.get(feature) == 'Стр':
        handled_feature = handle_text_feature(not_empty_factors_df[feature])
        #handled_feature = handle_text_feature(copy_df[feature])
    elif feature_types_dict.get(feature) == 'Кат':
        handled_feature = handle_cat_feature(not_empty_factors_df[feature])
        #handled_feature = handle_cat_feature(copy_df[feature])
    else:
        handled_feature = pd.DataFrame(not_empty_factors_df[feature])
        #handled_feature = pd.DataFrame(copy_df[feature])
    logger.debug(f'Feature: {feature}')
    handled_feature.columns = [str(col)+'_'+str(feature) for col in handled_feature.columns]
        
    trainset = pd.concat([trainset,handled_feature],axis=1)

2023-10-12 14:17:00.749 | DEBUG    | __main__:<module>:14 - Feature: Историческое наименование
2023-10-12 14:17:00.832 | DEBUG    | __main__:<module>:14 - Feature: ХК_Кат_01
2023-10-12 14:17:00.869 | DEBUG    | __main__:<module>:14 - Feature: Значение ХК_Кат_01
2023-10-12 14:17:00.899 | DEBUG    | __main__:<module>:14 - Feature: ХК_Кат_02
2023-10-12 14:17:00.928 | DEBUG    | __main__:<module>:14 - Feature: Значение ХК_Кат_02
2023-10-12 14:17:00.955 | DEBUG    | __main__:<module>:14 - Feature: ХК_Кат_03
2023-10-12 14:17:00.984 | DEBUG    | __main__:<module>:14 - Feature: Значение ХК_Кат_03
2023-10-12 14:17:01.011 | DEBUG    | __main__:<module>:14 - Feature: ХК_Кат_04
2023-10-12 14:17:01.038 | DEBUG    | __main__:<module>:14 - Feature: Значение ХК_Кат_04
2023-10-12 14:17:01.064 | DEBUG    | __main__:<module>:14 - Feature: ХК_Кат_05
2023-10-12 14:17:01.095 | DEBUG    | __main__:<module>:14 - Feature: Значение ХК_Кат_05
2023-10-12 14:17:01.121 | DEBUG    | __main__:<module>:14 - Feature: Х

In [32]:
trainset.isna().sum()

01_Историческое наименование                     57
02_Историческое наименование                     57
050_Историческое наименование                    57
100_Историческое наименование                    57
100h_Историческое наименование                   57
                                               ... 
кама_Значение ХК_Стр_01                        7607
нордмастер_Значение ХК_Стр_01                  7607
EmptyValue_ХК_Булево_01                           0
Шипы_ХК_Булево_01                                 0
Значение ХК_Булево_01_Значение ХК_Булево_01       0
Length: 767, dtype: int64

#### Сэмплируем классы, в которых всего 1 экземпляр и делим на обучающую и тестовую выборки

In [18]:
target.value_counts()

ID класса (ТАРГЕТ)
20000075    97
20000109    91
3584225     87
20000112    85
20000343    80
            ..
20000203     1
20000280     1
20000249     1
20000582     1
20000561     1
Name: count, Length: 645, dtype: int64

In [19]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
trainset_res, target_res = ros.fit_resample(trainset, target)
target_res.value_counts()

ID класса (ТАРГЕТ)
12388594    97
20000029    97
20000234    97
20000214    97
20000069    97
            ..
20000479    97
20000511    97
20000542    97
20000587    97
20000561    97
Name: count, Length: 645, dtype: int64

In [20]:
trainset_res.shape

(62565, 767)

In [24]:
trainset_res.isna().sum()

01_Историческое наименование                     493
02_Историческое наименование                     493
050_Историческое наименование                    493
100_Историческое наименование                    493
100h_Историческое наименование                   493
                                               ...  
кама_Значение ХК_Стр_01                        49922
нордмастер_Значение ХК_Стр_01                  49922
EmptyValue_ХК_Булево_01                            0
Шипы_ХК_Булево_01                                  0
Значение ХК_Булево_01_Значение ХК_Булево_01        0
Length: 767, dtype: int64

In [21]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 100)
reproduced_trainset = pca.fit_transform(np.transpose(trainset_res))

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [60]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(
    trainset_res, 
    target_res, 
    test_size=0.3, 
    random_state=42,
    #stratify=target    
) 

#### Обучим модель CatBoostClassifier на подготовленных данных

In [61]:
from catboost import CatBoostClassifier

In [68]:
model = CatBoostClassifier(loss_function='MultiClass', logging_level='Verbose')

#Нужно контролить переобоучение, правильно выставить learning_rate

grid_small = {'learning_rate': [0.01, 0.03, 0.09],
        'depth': [ 6, 8, 10]
        #'l2_leaf_reg': [1, 3, 5, 7, 9]
        }


grid_full = {'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09],
        'depth': [4, 6, 7, 8, 9, 10],
        #'early_stopping_rounds': True,
        'l2_leaf_reg': [1, 3, 5, 7, 9]
        }

grid_search_result = model.grid_search(
        grid_small,
        X=train_data,
        y=train_labels,
)

0:	learn: 5.7574141	test: 5.7582475	best: 5.7582475 (0)	total: 36.5s	remaining: 10h 7m 57s
1:	learn: 5.6694499	test: 5.6715185	best: 5.6715185 (1)	total: 1m 13s	remaining: 10h 12m 45s
2:	learn: 5.5638908	test: 5.5672835	best: 5.5672835 (2)	total: 1m 50s	remaining: 10h 10m 14s
3:	learn: 5.4813646	test: 5.4859392	best: 5.4859392 (3)	total: 2m 24s	remaining: 9h 59m 10s
4:	learn: 5.4053281	test: 5.4104532	best: 5.4104532 (4)	total: 2m 58s	remaining: 9h 51m 16s
5:	learn: 5.3249025	test: 5.3313308	best: 5.3313308 (5)	total: 3m 33s	remaining: 9h 48m 26s
6:	learn: 5.2597535	test: 5.2666543	best: 5.2666543 (6)	total: 4m 8s	remaining: 9h 46m 28s
7:	learn: 5.1865338	test: 5.1945012	best: 5.1945012 (7)	total: 4m 42s	remaining: 9h 43m 12s
8:	learn: 5.1291119	test: 5.1373489	best: 5.1373489 (8)	total: 5m 15s	remaining: 9h 39m 26s
9:	learn: 5.0640924	test: 5.0735234	best: 5.0735234 (9)	total: 5m 49s	remaining: 9h 36m 12s
10:	learn: 5.0081099	test: 5.0178710	best: 5.0178710 (10)	total: 6m 22s	remainin

88:	learn: 3.0590420	test: 3.0867121	best: 3.0867121 (88)	total: 50m 44s	remaining: 8h 39m 24s
89:	learn: 3.0479149	test: 3.0756980	best: 3.0756980 (89)	total: 51m 17s	remaining: 8h 38m 39s
90:	learn: 3.0346537	test: 3.0624587	best: 3.0624587 (90)	total: 51m 50s	remaining: 8h 37m 55s
91:	learn: 3.0249272	test: 3.0527812	best: 3.0527812 (91)	total: 52m 24s	remaining: 8h 37m 11s
92:	learn: 3.0158002	test: 3.0440843	best: 3.0440843 (92)	total: 52m 57s	remaining: 8h 36m 27s
93:	learn: 3.0044439	test: 3.0326442	best: 3.0326442 (93)	total: 53m 30s	remaining: 8h 35m 43s
94:	learn: 2.9959255	test: 3.0241757	best: 3.0241757 (94)	total: 54m 3s	remaining: 8h 34m 59s
95:	learn: 2.9855825	test: 3.0140081	best: 3.0140081 (95)	total: 54m 42s	remaining: 8h 35m 11s
96:	learn: 2.9733749	test: 3.0019499	best: 3.0019499 (96)	total: 55m 18s	remaining: 8h 34m 49s
97:	learn: 2.9668686	test: 2.9955227	best: 2.9955227 (97)	total: 55m 51s	remaining: 8h 34m 6s
98:	learn: 2.9539843	test: 2.9826754	best: 2.9826754

172:	learn: 2.3817713	test: 2.4165793	best: 2.4165793 (172)	total: 1h 38m 36s	remaining: 7h 51m 22s
173:	learn: 2.3768431	test: 2.4116908	best: 2.4116908 (173)	total: 1h 39m 9s	remaining: 7h 50m 43s
174:	learn: 2.3643890	test: 2.3994381	best: 2.3994381 (174)	total: 1h 39m 42s	remaining: 7h 50m 4s
175:	learn: 2.3598708	test: 2.3949371	best: 2.3949371 (175)	total: 1h 40m 15s	remaining: 7h 49m 25s
176:	learn: 2.3536646	test: 2.3886677	best: 2.3886677 (176)	total: 1h 40m 48s	remaining: 7h 48m 45s
177:	learn: 2.3470994	test: 2.3820274	best: 2.3820274 (177)	total: 1h 41m 22s	remaining: 7h 48m 6s
178:	learn: 2.3403103	test: 2.3755836	best: 2.3755836 (178)	total: 1h 41m 55s	remaining: 7h 47m 28s
179:	learn: 2.3363967	test: 2.3718367	best: 2.3718367 (179)	total: 1h 42m 33s	remaining: 7h 47m 14s
180:	learn: 2.3340857	test: 2.3695762	best: 2.3695762 (180)	total: 1h 43m 10s	remaining: 7h 46m 49s
181:	learn: 2.3320593	test: 2.3675856	best: 2.3675856 (181)	total: 1h 43m 46s	remaining: 7h 46m 26s
182

255:	learn: 1.9379812	test: 1.9779378	best: 1.9779378 (255)	total: 2h 26m 9s	remaining: 7h 4m 47s
256:	learn: 1.9361420	test: 1.9762065	best: 1.9762065 (256)	total: 2h 26m 43s	remaining: 7h 4m 10s
257:	learn: 1.9323239	test: 1.9725641	best: 1.9725641 (257)	total: 2h 27m 16s	remaining: 7h 3m 33s
258:	learn: 1.9240941	test: 1.9642174	best: 1.9642174 (258)	total: 2h 27m 49s	remaining: 7h 2m 56s
259:	learn: 1.9200600	test: 1.9602851	best: 1.9602851 (259)	total: 2h 28m 23s	remaining: 7h 2m 19s
260:	learn: 1.9144805	test: 1.9547418	best: 1.9547418 (260)	total: 2h 28m 56s	remaining: 7h 1m 42s
261:	learn: 1.9092731	test: 1.9495190	best: 1.9495190 (261)	total: 2h 29m 29s	remaining: 7h 1m 4s
262:	learn: 1.9075196	test: 1.9477365	best: 1.9477365 (262)	total: 2h 30m 2s	remaining: 7h 27s
263:	learn: 1.9019533	test: 1.9421712	best: 1.9421712 (263)	total: 2h 30m 43s	remaining: 7h 11s
264:	learn: 1.8972477	test: 1.9376020	best: 1.9376020 (264)	total: 2h 31m 19s	remaining: 6h 59m 42s
265:	learn: 1.8962

338:	learn: 1.6297174	test: 1.6720468	best: 1.6720468 (338)	total: 3h 14m	remaining: 6h 18m 17s
339:	learn: 1.6277758	test: 1.6701427	best: 1.6701427 (339)	total: 3h 14m 34s	remaining: 6h 17m 42s
340:	learn: 1.6272877	test: 1.6696791	best: 1.6696791 (340)	total: 3h 15m 8s	remaining: 6h 17m 7s
341:	learn: 1.6245243	test: 1.6669419	best: 1.6669419 (341)	total: 3h 15m 42s	remaining: 6h 16m 32s
342:	learn: 1.6209905	test: 1.6634565	best: 1.6634565 (342)	total: 3h 16m 15s	remaining: 6h 15m 56s
343:	learn: 1.6193370	test: 1.6618825	best: 1.6618825 (343)	total: 3h 16m 49s	remaining: 6h 15m 20s
344:	learn: 1.6166366	test: 1.6592877	best: 1.6592877 (344)	total: 3h 17m 22s	remaining: 6h 14m 44s
345:	learn: 1.6110868	test: 1.6536935	best: 1.6536935 (345)	total: 3h 17m 56s	remaining: 6h 14m 8s
346:	learn: 1.6083745	test: 1.6509579	best: 1.6509579 (346)	total: 3h 18m 32s	remaining: 6h 13m 37s
347:	learn: 1.6067410	test: 1.6493532	best: 1.6493532 (347)	total: 3h 19m 11s	remaining: 6h 13m 12s
348:	le

...

753:	learn: 0.8891290	test: 0.9372824	best: 0.9372824 (753)	total: 7h 15m 42s	remaining: 2h 22m 9s
754:	learn: 0.8877834	test: 0.9359838	best: 0.9359838 (754)	total: 7h 16m 15s	remaining: 2h 21m 34s
755:	learn: 0.8868900	test: 0.9350925	best: 0.9350925 (755)	total: 7h 16m 48s	remaining: 2h 20m 58s
756:	learn: 0.8861582	test: 0.9343260	best: 0.9343260 (756)	total: 7h 17m 21s	remaining: 2h 20m 23s
757:	learn: 0.8856829	test: 0.9338549	best: 0.9338549 (757)	total: 7h 18m 1s	remaining: 2h 19m 50s
758:	learn: 0.8848289	test: 0.9329706	best: 0.9329706 (758)	total: 7h 18m 41s	remaining: 2h 19m 17s
759:	learn: 0.8837845	test: 0.9320061	best: 0.9320061 (759)	total: 7h 19m 19s	remaining: 2h 18m 44s
760:	learn: 0.8814472	test: 0.9296094	best: 0.9296094 (760)	total: 7h 19m 57s	remaining: 2h 18m 10s
761:	learn: 0.8806478	test: 0.9288909	best: 0.9288909 (761)	total: 7h 20m 35s	remaining: 2h 17m 36s
762:	learn: 0.8797565	test: 0.9280116	best: 0.9280116 (762)	total: 7h 21m 8s	remaining: 2h 17m 1s
763:

836:	learn: 0.8148580	test: 0.8632767	best: 0.8632767 (836)	total: 8h 5m 6s	remaining: 1h 34m 28s
837:	learn: 0.8132883	test: 0.8616486	best: 0.8616486 (837)	total: 8h 5m 39s	remaining: 1h 33m 53s
838:	learn: 0.8118930	test: 0.8601874	best: 0.8601874 (838)	total: 8h 6m 12s	remaining: 1h 33m 18s
839:	learn: 0.8111535	test: 0.8594349	best: 0.8594349 (839)	total: 8h 6m 45s	remaining: 1h 32m 43s
840:	learn: 0.8106298	test: 0.8589500	best: 0.8589500 (840)	total: 8h 7m 19s	remaining: 1h 32m 7s
841:	learn: 0.8094785	test: 0.8578348	best: 0.8578348 (841)	total: 8h 7m 52s	remaining: 1h 31m 32s
842:	learn: 0.8087469	test: 0.8571261	best: 0.8571261 (842)	total: 8h 8m 29s	remaining: 1h 30m 58s
843:	learn: 0.8076836	test: 0.8559981	best: 0.8559981 (843)	total: 8h 9m 8s	remaining: 1h 30m 24s
844:	learn: 0.8066544	test: 0.8549840	best: 0.8549840 (844)	total: 8h 9m 48s	remaining: 1h 29m 50s
845:	learn: 0.8056088	test: 0.8539294	best: 0.8539294 (845)	total: 8h 10m 26s	remaining: 1h 29m 16s
846:	learn: 

920:	learn: 0.7396521	test: 0.7878643	best: 0.7878643 (920)	total: 8h 55m 6s	remaining: 45m 53s
921:	learn: 0.7389883	test: 0.7872045	best: 0.7872045 (921)	total: 8h 55m 45s	remaining: 45m 19s
922:	learn: 0.7377433	test: 0.7859501	best: 0.7859501 (922)	total: 8h 56m 24s	remaining: 44m 44s
923:	learn: 0.7368932	test: 0.7851056	best: 0.7851056 (923)	total: 8h 56m 57s	remaining: 44m 9s
924:	learn: 0.7363368	test: 0.7845552	best: 0.7845552 (924)	total: 8h 57m 30s	remaining: 43m 34s
925:	learn: 0.7352411	test: 0.7834106	best: 0.7834106 (925)	total: 8h 58m 4s	remaining: 42m 59s
926:	learn: 0.7342267	test: 0.7823694	best: 0.7823694 (926)	total: 8h 58m 38s	remaining: 42m 25s
927:	learn: 0.7334464	test: 0.7816434	best: 0.7816434 (927)	total: 8h 59m 11s	remaining: 41m 50s
928:	learn: 0.7319781	test: 0.7801054	best: 0.7801054 (928)	total: 8h 59m 44s	remaining: 41m 15s
929:	learn: 0.7316029	test: 0.7797292	best: 0.7797292 (929)	total: 9h 22s	remaining: 40m 40s
930:	learn: 0.7308530	test: 0.7789979

5:	learn: 4.5809863	test: 4.5937527	best: 4.5937527 (5)	total: 3m 26s	remaining: 9h 29m 7s
6:	learn: 4.4468245	test: 4.4611823	best: 4.4611823 (6)	total: 4m 5s	remaining: 9h 40m 56s
7:	learn: 4.2985764	test: 4.3151034	best: 4.3151034 (7)	total: 4m 45s	remaining: 9h 49m 26s
8:	learn: 4.1973755	test: 4.2145195	best: 4.2145195 (8)	total: 5m 23s	remaining: 9h 54m 19s
9:	learn: 4.0950343	test: 4.1136460	best: 4.1136460 (9)	total: 6m 2s	remaining: 9h 58m 2s


KeyboardInterrupt: 

In [ ]:
model.fit(train_data, train_labels)
preds_labels = model.predict(test_data)

In [73]:
model_2 = CatBoostClassifier(loss_function='MultiClass', logging_level='Verbose', learning_rate = 0.3, depth = 4)

#Нужно контролить переобоучение, правильно выставить learning_rate, метрика и функция потерь
model_2.fit(train_data, train_labels)
preds_labels = model_2.predict(test_data)

0:	learn: 4.4773927	total: 27.6s	remaining: 7h 40m 22s
1:	learn: 4.1295189	total: 54.8s	remaining: 7h 35m 20s
2:	learn: 3.5977834	total: 1m 21s	remaining: 7h 33m 10s


KeyboardInterrupt: 

In [71]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(test_labels, preds_labels)
report = classification_report(test_labels, preds_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

ValueError: Found input variables with inconsistent numbers of samples: [18770, 437]

#### Удалим бесполезные факторы которые состоят из 1 уникального значения

In [ ]:
#Делаем это после кодирования категориалььных переменных
#for column in factors_df.columns:
#    unique_values_count = factors_df[column].drop_duplicates().size
#    if unique_values_count == 1:
#        new_factors_df = factors_df.drop(column, axis = 1)

In [ ]:
#new_factors_df.info()

In [ ]:
#new_factors_df['Наименование терминального класса'].value_counts()

* Удалить колонки для неполного соответствия: если есть [1,2,3,4] - удалить одну, чтобы не было зависимости
* Удалить строки без характеристик
* Отбор на основе важности признаков